# 全连接层

In [1]:
import numpy as np


In [2]:
class FullyConnectedLayer(object):
    """A simple fully-connected NN layer.
    Args:
        num_inputs (int): 输入参数的个数.
        layer_size (int): 神经元层数.
        activation_function (callable): The activation function for this layer.
    Attributes:
        W (ndarray): The weight values for each input.
        b (ndarray): The bias value, added to the weighted sum.
        size (int): The layer size / number of neurons.
        activation_function (callable): The activation function computing the neuron's output.
        x (ndarray): The last provided input vector, stored for backpropagation.
        y (ndarray): The corresponding output, also stored for backpropagation.
        derivated_activation_function (callable): The corresponding derivated function for backpropagation.
        dL_dW (ndarray): The derivative of the loss, with respect to the weights W.
        dL_db (ndarray): The derivative of the loss, with respect to the bias b.
    """

    def __init__(self, num_inputs, layer_size, activation_function, derivated_activation_function=None):
        super().__init__()

        # 使用正态分布 
        # Randomly initializing the weight vector and the bias value (using a normal distribution this time):
        self.W = np.random.standard_normal((num_inputs, layer_size))
        self.b = np.random.standard_normal(layer_size)
        self.size = layer_size

        self.activation_function = activation_function
        self.derivated_activation_function = derivated_activation_function
        self.x, self.y = None, None
        self.dL_dW, self.dL_db = None, None

    def forward(self, x):
        """
        Forward the input vector through the layer, returning its activation vector.
        Args:
            x (ndarray): The input vector, of shape `(batch_size, num_inputs)`
        Returns:
            activation (ndarray): The activation value, of shape `(batch_size, layer_size)`.
        """
        z = np.dot(x, self.W) + self.b
        self.y = self.activation_function(z)
        # (we store the input and output values for back-propagation)
        self.x = x
        return self.y

    def backward(self, dL_dy):
        """
        Back-propagate the loss, computing all the derivatives, storing those w.r.t. the layer parameters,
        and returning the loss w.r.t. its inputs for further propagation.
        Args:
            dL_dy (ndarray): The loss derivative w.r.t. the layer's output (dL/dy = l'_{k+1}).
        Returns:
            dL_dx (ndarray): The loss derivative w.r.t. the layer's input (dL/dx).
        """
        dy_dz = self.derivated_activation_function(self.y)  # = f'
        dL_dz = (dL_dy * dy_dz)  # dL/dz = dL/dy * dy/dz = l'_{k+1} * f'
        dz_dw = self.x.T
        dz_dx = self.W.T
        # dz/db = d(W.x + b)/db = 0 + db/db = "ones"-vector
        dz_db = np.ones(dL_dy.shape[0])

        # Computing the derivatives with respect to the layer's parameters, and storing them for opt. optimization:
        self.dL_dW = np.dot(dz_dw, dL_dz)
        self.dL_db = np.dot(dz_db, dL_dz)

        # Computing the derivative with respect to the input, to be passed to the previous layers (their `dL_dy`):
        dL_dx = np.dot(dL_dz, dz_dx)
        return dL_dx

    def optimize(self, epsilon):
        """
        Optimize the layer's parameters, using the stored derivative values.
        Args:
            epsilon (float): The learning rate.
        """
        self.W -= epsilon * self.dL_dW
        self.b -= epsilon * self.dL_db


In [8]:
def relu_function(y): return np.maximum(
    y, 0)    # Defining our activation function




In [9]:
# Fixing the seed for the random number generation, to get reproducable results.
np.random.seed(42)

x1 = np.random.uniform(-1, 1, 2).reshape(1, 2)
# > [[-0.25091976  0.90142861]]
x2 = np.random.uniform(-1, 1, 2).reshape(1, 2)
# > [[0.46398788 0.19731697]]

layer = FullyConnectedLayer(num_inputs=2, layer_size=3,
                            activation_function=relu_function, derivated_activation_function=None)


In [10]:
print(x1)
print(x2)


[[-0.25091976  0.90142861]]
[[0.46398788 0.19731697]]


In [13]:
out1 = layer.forward(x1)
print(out1)
out2 = layer.forward(x2)
print(out2)
# > [[0.         0.         1.08175419]]


[[0.28712364 0.         0.33478571]]
[[0.         0.         1.08175419]]


In [15]:
# ... or together:
x12 = np.concatenate((x1, x2))  # stack of input vectors, of shape `(2, 2)`
out12 = layer.forward(x12)
print(out12)


[[0.28712364 0.         0.33478571]
 [0.         0.         1.08175419]]
